In [4]:
import pandas as pd
import time
import numpy as np
import plotly.express as px

In [30]:
import plotly.io as pio

pio.renderers.default = "vscode"  # or "vscode" if using VS Code notebooks

template = pio.templates["plotly_dark"]

# Backgrounds
template.layout.plot_bgcolor = "#2B2B2B"   # inside the axes # type: ignore
template.layout.paper_bgcolor = "#2B2B2B"  # around the plot # type: ignore

# X axis
template.layout.xaxis.color = "#A9B7C6"    # tick labels + title # type: ignore
template.layout.xaxis.gridcolor = "#7B7E82" # type: ignore
template.layout.xaxis.showline = False # type: ignore
template.layout.xaxis.linecolor = "#A9B7C6" # type: ignore
template.layout.xaxis.tickcolor = "#A9B7C6"  # color of tick lines # type: ignore
template.layout.xaxis.zeroline = False # type: ignore
template.layout.xaxis.zerolinecolor = "#A9B7C6" # type: ignore

# Y axis
template.layout.yaxis.color = "#A9B7C6" # type: ignore
template.layout.yaxis.gridcolor = "#7B7E82" # type: ignore
template.layout.yaxis.showline = False # type: ignore
template.layout.yaxis.linecolor = "#A9B7C6" # type: ignore
template.layout.yaxis.tickcolor = "#A9B7C6" # type: ignore
template.layout.yaxis.zeroline = False # type: ignore
template.layout.yaxis.zerolinecolor = "#A9B7C6" # type: ignore

# Register as new template
pio.templates["custom_dark"] = template
pio.templates.default = "custom_dark"

In [5]:
import os
print(os.getcwd())

/Users/alecmitchell-thomson/Desktop/Alpha Fund Bootcamp/Project/Alpha-Fund-Project/data


In [6]:
TICKERS = [
    "SPY", "QQQ", "IWM",                                                            # Core market index ETFs
    "AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "META", "GOOGL", "JPM", "XOM", "WMT",   # Mega-cap stocks
    "AMD", "AVGO", "CRM", "SHOP",                                                   # High-growth volatile tech stocks
    "XLK", "XLF", "XLE", "XLY", "XLV",                                              # Sector ETFs
    "PLTR", "ROKU", "COIN", "TWLO", "UBER"                                          # Emerging volatile tech stocks
]

In [7]:
def raw_file_path(ticker):
    return f"raw_5min/{ticker}_2023-11-01_to_2025-11-23.csv"

## **1. Concatenate all of the individual CSV's into one dataframe**

In [10]:
dfs = []
for ticker in TICKERS:
    df_t = pd.read_csv(raw_file_path(ticker), parse_dates=['timestamp'])
    df_t['ticker'] = ticker
    dfs.append(df_t)

df_raw = pd.concat(dfs, ignore_index=True)
df_raw = df_raw.sort_values(['ticker', 'timestamp']).reset_index(drop=True)
df_raw

,timestamp,open,high,low,close,volume,vwap,num_trades,ticker
0,2023-11-27 09:00:00+00:00,189.6000,189.6000,189.2000,189.2500,2044.0,189.2788,96,AAPL
1,2023-11-27 09:10:00+00:00,189.2700,189.4100,189.2700,189.4000,3044.0,189.3226,56,AAPL
2,2023-11-27 09:15:00+00:00,189.4000,189.4000,189.2800,189.2800,3144.0,189.2938,48,AAPL
3,2023-11-27 09:20:00+00:00,189.3000,189.3000,189.3000,189.3000,521.0,189.2983,12,AAPL
4,2023-11-27 09:25:00+00:00,189.2700,189.3000,189.2500,189.3000,3404.0,189.2751,50,AAPL
...,...,...,...,...,...,...,...,...,...
1983759,2025-11-21 22:55:00+00:00,117.0640,117.0640,117.0640,117.0640,100.0,117.0640,1,XOM
1983760,2025-11-21 23:00:00+00:00,116.8107,116.8107,116.8107,116.8107,100.0,116.8107,1,XOM
1983761,2025-11-21 23:05:00+00:00,117.0482,117.0482,117.0482,117.0482,100.0,117.0482,1,XOM
1983762,2025-11-21 23:10:00+00:00,117.1900,117.1900,117.1900,117.1900,370.0,117.1905,2,XOM


## **2. Filter to regular trading hours**
- US equities RTH is **9.30-16.00**
- Pre-market and after hours regimes have completely different patterns, and will not include stop-loss microstructure so is not useful here

In [15]:
df_raw = df_raw.set_index('timestamp')
df_raw = df_raw.between_time('09:30', '16:00') # filter for regular trading hours
df_raw = df_raw.reset_index()
df_RTH = df_raw[df_raw['timestamp'].dt.dayofweek < 5]  # Remove weekends if any
df_RTH["date"] = df_RTH["timestamp"].dt.date
df_RTH

,timestamp,open,high,low,close,volume,vwap,num_trades,ticker,date
0,2023-11-27 09:30:00+00:00,189.300,189.300,189.300,189.3000,515.0,189.2919,19,AAPL,2023-11-27
1,2023-11-27 09:35:00+00:00,189.310,189.310,189.250,189.2500,544.0,189.2761,25,AAPL,2023-11-27
2,2023-11-27 09:45:00+00:00,189.250,189.250,189.250,189.2500,707.0,189.2497,24,AAPL,2023-11-27
3,2023-11-27 09:55:00+00:00,189.300,189.300,189.260,189.2800,3101.0,189.2824,56,AAPL,2023-11-27
4,2023-11-27 10:00:00+00:00,189.270,189.270,189.270,189.2700,204.0,189.2703,4,AAPL,2023-11-27
...,...,...,...,...,...,...,...,...,...,...
801452,2025-11-21 15:40:00+00:00,116.365,116.460,116.250,116.4150,107103.0,116.3544,1935,XOM,2025-11-21
801453,2025-11-21 15:45:00+00:00,116.440,116.725,116.340,116.5750,135189.0,116.6059,2514,XOM,2025-11-21
801454,2025-11-21 15:50:00+00:00,116.575,116.710,116.445,116.6900,119133.0,116.5369,2505,XOM,2025-11-21
801455,2025-11-21 15:55:00+00:00,116.680,116.700,116.300,116.3100,149766.0,116.5261,2211,XOM,2025-11-21


## **3. Handle missing bars**
- check the expected number of bars per day
- if num(bars) =< 75, drop the entire day for that ticker (trying to interpolate large gaps corrupts volatility)

In [68]:
bar_counts = df_RTH.groupby(['ticker', 'date']).size().rename('n_bars')
good_days = bar_counts[bar_counts >= 70].index
good_days_df = good_days.to_frame(index=False)


df_good_days = pd.merge(df_RTH, good_days_df, on=['ticker', 'date'], how='inner')
df_good_days

# fig = px.bar(df_bars, x='index', y='count', labels={'index': 'Number of 5-min Bars per Day', 0: 'Count of Days'},
#                    title='Distribution of 5-min Bars per Day Across Tickers')
# fig.show()

,timestamp,open,high,low,close,volume,vwap,num_trades,ticker,date
0,2023-11-29 09:30:00+00:00,190.150,190.25,190.140,190.2500,7436.0,190.1900,141,AAPL,2023-11-29
1,2023-11-29 09:40:00+00:00,190.160,190.16,190.160,190.1600,1696.0,190.1408,51,AAPL,2023-11-29
2,2023-11-29 09:45:00+00:00,190.170,190.20,190.170,190.2000,3189.0,190.1975,42,AAPL,2023-11-29
3,2023-11-29 09:50:00+00:00,190.290,190.29,190.280,190.2900,1764.0,190.2869,24,AAPL,2023-11-29
4,2023-11-29 09:55:00+00:00,190.250,190.30,190.250,190.3000,1159.0,190.2794,34,AAPL,2023-11-29
...,...,...,...,...,...,...,...,...,...,...
434385,2025-10-31 15:40:00+00:00,113.960,114.01,113.780,113.9250,113456.0,113.9067,2334,XOM,2025-10-31
434386,2025-10-31 15:45:00+00:00,113.940,113.94,113.210,113.3050,248170.0,113.4934,4333,XOM,2025-10-31
434387,2025-10-31 15:50:00+00:00,113.290,113.30,113.000,113.2680,133944.0,113.1499,2341,XOM,2025-10-31
434388,2025-10-31 15:55:00+00:00,113.270,113.48,113.205,113.4200,95896.0,113.3483,1568,XOM,2025-10-31


In [72]:
days_per_ticker = (
    df_good_days.groupby("ticker")["date"]
            .nunique()
            .sort_values()
)

good_tickers = days_per_ticker[days_per_ticker > 400].index

df_filtered = df_good_days[df_good_days['ticker'].isin(good_tickers)].copy()
df_filtered['missing_bars_day'] = df_filtered.groupby(['ticker', 'date'])['close'].transform(
    lambda s: 79 - len(s)
)
df_filtered

,timestamp,open,high,low,close,volume,vwap,num_trades,ticker,date,missing_bars_day
0,2023-11-29 09:30:00+00:00,190.1500,190.2500,190.1400,190.2500,7436.0,190.1900,141,AAPL,2023-11-29,8
1,2023-11-29 09:40:00+00:00,190.1600,190.1600,190.1600,190.1600,1696.0,190.1408,51,AAPL,2023-11-29,8
2,2023-11-29 09:45:00+00:00,190.1700,190.2000,190.1700,190.2000,3189.0,190.1975,42,AAPL,2023-11-29,8
3,2023-11-29 09:50:00+00:00,190.2900,190.2900,190.2800,190.2900,1764.0,190.2869,24,AAPL,2023-11-29,8
4,2023-11-29 09:55:00+00:00,190.2500,190.3000,190.2500,190.3000,1159.0,190.2794,34,AAPL,2023-11-29,8
...,...,...,...,...,...,...,...,...,...,...,...
417217,2025-11-21 15:40:00+00:00,385.5600,388.8600,384.2400,387.5231,1787242.0,386.3321,46055,TSLA,2025-11-21,0
417218,2025-11-21 15:45:00+00:00,387.5851,390.2391,386.2100,389.7200,1686153.0,388.7163,39877,TSLA,2025-11-21,0
417219,2025-11-21 15:50:00+00:00,389.6750,390.1000,386.5000,388.9600,1114335.0,388.1680,30916,TSLA,2025-11-21,0
417220,2025-11-21 15:55:00+00:00,389.0600,389.0600,385.5109,385.8300,1044840.0,386.9534,30918,TSLA,2025-11-21,0


## **4. Check validity of cleaned dataset**

In [90]:
df_filtered["timestamp"] = pd.to_datetime(df_filtered["timestamp"])
df_filtered["date"] = df_filtered["timestamp"].dt.date

bars_per_day = (
    df_filtered.groupby(["ticker", "date"])
      .size()
      .rename("n_bars")
)

# Quick summary
print(bars_per_day.describe())
print(bars_per_day.value_counts().sort_index())

count    4114.000000
mean       77.454545
std         2.266469
min        70.000000
25%        77.000000
50%        79.000000
75%        79.000000
max        79.000000
Name: n_bars, dtype: float64
n_bars
70      60
71      84
72      96
73     129
74     140
75     198
76     287
77     386
78     575
79    2159
Name: count, dtype: int64


In [83]:
date_range = df_filtered.groupby("ticker")["date"].agg(["min","max","nunique"])
print(date_range)

               min         max  nunique
ticker                                 
AAPL    2023-11-29  2025-11-21      459
AMD     2023-12-07  2025-11-21      445
AMZN    2023-11-27  2025-11-21      439
IWM     2023-11-27  2025-11-21      492
NVDA    2023-11-29  2025-11-21      423
PLTR    2023-11-27  2025-11-21      461
QQQ     2023-11-27  2025-11-21      469
SPY     2023-11-27  2025-11-21      485
TSLA    2023-11-27  2025-11-21      441


In [78]:
bars_per_day = df_filtered.groupby(["ticker","date"]).size()
print(bars_per_day.min(), bars_per_day.max(), bars_per_day.median())

70 79 79.0


#### Check that most tickers exist on most timestamps
- just counting the number of tickers that each timestamp has (should be close to 9 for 9 tickers)

In [93]:
counts_by_timestamp = df_filtered.groupby("timestamp")["ticker"].nunique()
print(counts_by_timestamp.describe())

count    39498.000000
mean         8.067446
std          1.496384
min          1.000000
25%          8.000000
50%          9.000000
75%          9.000000
max          9.000000
Name: ticker, dtype: float64


## **5. Save cleaned dataset to processed file**

In [97]:
out_path = "processed/cleaned_5_min_data.csv"

column_order = ["ticker", "timestamp", "date", "open", "high", "low", "close", "volume", "vwap", "num_trades", "missing_bars_day"]
df_clean = df_filtered[column_order]
df_clean.to_csv(out_path, index=False)